In [ ]:
n_classes = 3

# Location of data
datadir = '/content/drive/MyDrive/Interpretable Classifier Data/Vanilla Classifier Data/'
traindir = datadir + 'train/'
validdir = datadir + 'valid/'
testdir = datadir + 'test/'

# Change to fit hardware
batch_size = 10

train_on_gpu = True

# Importing Libraries

In [ ]:
from PIL import Image
import cv2
import glob
import random
import torch.nn as nn
import torch
from torch.utils.data import Dataset
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torchvision.transforms as transforms

# Loading data


In [ ]:
import torchvision.transforms as transforms

transform_img = transforms.Compose([
  transforms.Resize(size=256),
  transforms.CenterCrop(size=224),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Location of data
datadir = '/content/drive/MyDrive/Interpretable Classifier Data/Vanilla Classifier Data/'
traindir = datadir + 'train/'
validdir = datadir + 'valid/'
testdir = datadir + 'test/'

In [ ]:
# Datasets from each folder
data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=transform_img),
    'val':
    datasets.ImageFolder(root=validdir, transform=transform_img),
    'test':
    datasets.ImageFolder(root=testdir, transform=transform_img)
}

# Dataloader iterators
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['val'], batch_size=1, shuffle=True),
    'test': DataLoader(data['test'], batch_size=1, shuffle=True)
}

# Utility Functions

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
def evaluate(model,val_loader,epoch,criterion):

  acc = 0
  epoch_loss = 0.0
  criterion = nn.CrossEntropyLoss()

  for img,label in val_loader:

    model.eval()
  
    xb = img.to('cuda')
    label = label.to('cuda').long()

    yb = model(xb)

    loss = criterion(yb, label)
    epoch_loss += loss.item()

    pred = F.softmax(yb, dim=1)

    _, preds  = torch.max(yb, dim=1)

    preds = preds.cpu()
    label = label.cpu()

    temp = (np.array( preds == label )).sum()
    acc += temp

  print("Accuracy = ",acc*100/len(val_loader))
  print("------------")

  return {"Accuracy":acc*100/len(val_loader),"Loss":epoch_loss/len(val_loader)}

In [ ]:
%mkdir checkpoint best_model

In [ ]:
import shutil
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

In [ ]:
import torch.nn.functional as F

def fit(epochs, lr, model, train_loader, val_loader,criterion,opt_func=torch.optim.Adam,checkpoint_path="/content/checkpoint/current_checkpoint.pt", best_model_path="/content/best_model/best_model.pt"):

    epoch_data = {}
    optim = opt_func(model.parameters(),lr=lr,weight_decay=0.0005)

    valid_loss_min = np.Inf

    for epoch in range(epochs):

        # Training Phase 
        model.train()
        epoch_loss = 0.0

        for step, batch_data in enumerate(train_loader):

            # Get the inputs and labels
            inputs = batch_data[0].to('cuda')
            labels = batch_data[1].to('cuda').long()

            # Forward propagation
            outputs = model(inputs)

            # Loss computation
            loss = criterion(outputs, labels)

            # Backpropagation
            optim.zero_grad()
            loss.backward()
            optim.step()

            # Keep track of loss for current epoch
            epoch_loss += loss.item()

        epoch_data[epoch+1] = evaluate(model,val_loader,epoch+1,criterion)
        val_loss = epoch_data[epoch+1]["Loss"]

        print("Epoch number:",epoch+1," - Training Loss = " , epoch_loss / len(train_loader) , " Valiadtion Loss = ", val_loss )

        wandb.log( { "Training Loss" : epoch_loss / len(train_loader) , 
                     "Validation Loss" : val_loss } )

        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': val_loss,
            'state_dict': model.state_dict(),
            'optimizer': optim.state_dict(),
        }

        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        if val_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,val_loss))
            # save checkpoint as best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = val_loss

        print("------------")

    return epoch_data

# Model

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)
model.classifier[6] = nn.Linear(4096,3)

In [ ]:
model = model.to('cuda')

# Training

In [ ]:
num_epochs = 100
opt_func = torch.optim.Adam
lr = 0.0001
criterion = nn.CrossEntropyLoss()

## Wandb

In [ ]:
!pip3 install wandb
import wandb
wandb.login()
wandb.init(project="AlexNet")

In [ ]:
epoch_data = fit(num_epochs, lr, model, dataloaders['train'], dataloaders['val'] , criterion)

# Loading Model

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)
model.classifier[6] = nn.Linear(4096,3)
model = model.to('cuda')

In [ ]:
# define optimzer
optimizer = torch.optim.Adam(model.parameters())

# define checkpoint saved path
ckp_path = "/content/drive/MyDrive/Interpretable Classifier Data/Vanilla models/alexnet-final.pt"

In [ ]:
model, optimizer, start_epoch, valid_loss_min = load_ckp(ckp_path, model, optimizer)

In [ ]:
print("start_epoch = ", start_epoch)
print("valid_loss_min = ", valid_loss_min)

# Testing

In [ ]:
y_pred = []
y_true = []

In [ ]:
acc = 0
epoch_loss = 0.0
criterion = nn.CrossEntropyLoss()

for img, label in dataloaders['test']:

  model.eval()
  
  xb = img.to('cuda')
  label = label.to('cuda').long()

  yb = model(xb)

  pred = F.softmax(yb, dim=1)
  _, preds  = torch.max(yb, dim=1)

  preds = preds.cpu()
  label = label.cpu()

  y_pred.append(int(preds))
  y_true.append(int(label))

  temp = (np.array( preds == label )).sum()
  acc += temp

print("Accuracy = ",acc*100/len(dataloaders['test']))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)*100

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))